In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "fill out your API key"

In [ ]:
! pip install --quiet \
  "langchain>=0.2" \
  "langgraph>=0.2" \
  faiss-cpu \
  openai \
  tiktoken \
  pydantic \
  typing-extensions \
  langchain_community

In [ ]:
! pip install langchain_openai

In [ ]:
# =============================================================================
# - 한 개의 벡터 DB(해결기준+법률+사례 혼합)에 대해 단일 RetrievalQA 수행
# =============================================================================

from __future__ import annotations
import os
from typing import List, Dict, Any
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# -------------------------
# 환경/모델
# -------------------------
if not os.environ.get("OPENAI_API_KEY"):
    raise RuntimeError("OPENAI_API_KEY를 먼저 설정하세요. 예) os.environ['OPENAI_API_KEY']='sk-...'")

openai_llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    # streaming=False  # 필요하면 True로 바꾸고 콜백 추가
)
emb = OpenAIEmbeddings(model="text-embedding-3-large")

# -------------------------
# 혼합 인덱스 경로 (수정)
#    * 해결기준+법률+사례가 하나의 FAISS로 합쳐진 인덱스
# -------------------------
DB_MIXED = "fill out DB path"  # ← 네 경로로 수정
vmix = FAISS.load_local(DB_MIXED, emb, allow_dangerous_deserialization=True)

# 단일 리트리버 (구조화 RAG와 유사한 MMR 세팅)
retriever = vmix.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2, "fetch_k": 7},
)

# -------------------------
# 프롬프트 (구조화 RAG 조건과 동일한 규율)
#    - 해결기준 > 법률 > 사례 우선
#    - 외부 지식 금지, 불충분 시 "I don't know"
#    - 최종 요약은 미괄식(결론을 문단 끝부분), 3~5문장
# -------------------------
single_stuff_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""

  "당신은 소비자분쟁 전문가입니다. 주어진 질문의 요지를 파악하고 적절한 대답을 해주세요.\n"
    "주어진 질문에 대해서 사전에 반드시 주어진 문서만을 토대로 기준 지침을 파악한 후, "
    "법률적 조언이 필요한 대답은 제시된 '민법', '소비자보호법', '전자상거래법'의 문서를 참조하여 답변을 구성해주세요.\n"
    "만약 사용자가 사례를 요청할 시, 제시된 '분쟁조정사례' 문서를 참조하여 사례의 구체적 내용을 함께 말해주세요.\n"
    "적절한 답변이 불가능할 시 'I don't know'로 대답해 주세요.\n\n"
    "추상적 단어의 처리방법\n"
    " - 비공개 장소: 화장실, 개인 객실 등\n"
    " - 민감정보: 개인 주민등록번호, 주소, 연락처 등\n"
    "위와 같이 질문의 추상적인 단어는 너가 보유한 지식체계 내에서 구체화하여 고려한 다음에 답변을 부탁해."


--- 컨텍스트 시작 ---
{context}
--- 컨텍스트 끝 ---

[질문]
{question}

"""
)

# -------------------------
# 단일 체인
# -------------------------
single_chain = RetrievalQA.from_chain_type(
    llm=openai_llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type="stuff",
    chain_type_kwargs={"prompt": single_stuff_prompt},
)

# -------------------------
# 실행 헬퍼 (평가 친화 포맷)
# -------------------------
def run_single(question: str, *, max_contexts: int = 5) -> Dict[str, Any]:
    """
    단일 RAG 실행 → {
        'answer': 모델 출력(한 문단 답변),
        'sources': 사용된 소스 문서 리스트,
        'contexts': Ragas용 컨텍스트 문자열 리스트(상위 n개)
    }
    """
    out = single_chain.invoke({"query": question})
    answer = (out.get("result") or "").strip()
    srcs = out.get("source_documents", []) or []
    contexts = [(getattr(d, "page_content", "") or "") for d in srcs][:max_contexts]
    return {
        "answer": answer,
        "sources": srcs,
        "contexts": contexts,
    }

In [ ]:
if __name__ == "__main__":
    q = "가전제품 렌탈에 제품 관리 조항까지 있었음에도 불구하구 렌탈업체의 제품관리 부실 이행으로 환불받고자하는데 위약금 면제가 되나요?"
    result = run_single(q)
    print("\n=== ANSWER ===\n", result["answer"])
    # Ragas/BERTScore용 contexts 확인
    print(f"\n[contexts used: {len(result['contexts'])}]")
    # 소스 확인 필요 시:
    # for i, d in enumerate(result["sources"], 1):
    #     print(f"\n--- SOURCE {i} ---\n", (d.page_content or "")[:800])